### 作者：小z
### 公众号：数据不吹牛
### 更多有趣干货，不来看看？

In [ ]:
import pandas as pd
import os
import warnings

In [36]:
warnings.filterwarnings("ignore")

In [108]:
os.chdir('C:\\Users\\order')

### 1、数据概览

In [81]:
df = pd.read_excel('同期群订单数据.xlsx')
df.head()

,平台,店铺名称,客户昵称,付款时间,订单状态,支付金额,购买数量,省份
0,线上,小z杂货铺,入倩出入深,2019-09-01 00:10:04,交易成功,15.2,1,江苏省
1,线上,小z杂货铺,愛hya爱,2019-09-01 00:14:52,交易成功,8.4,1,广东省
2,线上,小z杂货铺,象95象大,2019-09-01 02:17:15,交易成功,8.4,1,辽宁省
3,线上,小z杂货铺,卡哇伊氛十,2019-09-01 03:37:28,交易成功,22.0,1,广西壮族自治区
4,线上,小z杂货铺,一只羊哈阿,2019-09-01 08:53:50,交易成功,85.0,1,辽宁省


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42713 entries, 0 to 42712
Data columns (total 8 columns):
平台      42713 non-null object
店铺名称    42713 non-null object
客户昵称    42713 non-null object
付款时间    40339 non-null datetime64[ns]
订单状态    42713 non-null object
支付金额    42713 non-null float64
购买数量    42713 non-null int64
省份      42713 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 2.6+ MB


### 2、数据清洗

In [83]:
df.loc[df['付款时间'].isnull(),:].head()

,平台,店铺名称,客户昵称,付款时间,订单状态,支付金额,购买数量,省份
40339,线上,小z杂货铺,爱购物nx,NaT,交易失败,97.8,1,浙江省
40340,线上,小z杂货铺,975ay,NaT,交易失败,117.3,2,浙江省
40341,线上,小z杂货铺,101呆阿,NaT,交易失败,144.5,2,新疆维吾尔自治区
40342,线上,小z杂货铺,489bt,NaT,交易失败,92.7,1,江苏省
40343,线上,小z杂货铺,姚琳儿姚姚,NaT,交易失败,8.4,1,广东省


In [84]:
df.loc[df['付款时间'].isnull(),:]['订单状态'].value_counts()

交易失败    2374
Name: 订单状态, dtype: int64

In [85]:
df.loc[df['付款时间'].isnull() == False,:]['订单状态'].value_counts()

交易成功    40339
Name: 订单状态, dtype: int64

In [110]:
order = df.loc[df['付款时间'].isnull() == False,:]
order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 40338
Data columns (total 8 columns):
平台      40339 non-null object
店铺名称    40339 non-null object
客户昵称    40339 non-null object
付款时间    40339 non-null datetime64[ns]
订单状态    40339 non-null object
支付金额    40339 non-null float64
购买数量    40339 non-null int64
省份      40339 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 2.8+ MB


### 3、单月实现

In [111]:
order['时间标签'] = order['付款时间'].astype(str).str[:7]
order['时间标签'].value_counts().sort_index()

2019-09    2201
2019-10    8096
2019-11    6050
2019-12    6760
2020-01    7443
2020-02    9789
Name: 时间标签, dtype: int64

#### 计算2019年10月客户情况

In [103]:
month = '2019-10'
sample = order.loc[order['时间标签'] == month,:]
print('10月订单数量：',len(sample))
sample.head()
sample_c = sample.groupby('客户昵称')['支付金额'].sum().reset_index()
print('10月客户数量：',len(sample_c))
sample_c.head()

10月订单数量： 8096
10月客户数量： 7336


,客户昵称,支付金额
0,0000栗,4.2
1,000ab,16.8
2,000al,16.8
3,000bb,16.8
4,000il,16.8


In [89]:
history = order.loc[order['时间标签'] == '2019-09',:]
history.head()

,平台,店铺名称,客户昵称,付款时间,订单状态,支付金额,购买数量,省份,时间标签
0,线上,小z杂货铺,入倩出入深,2019-09-01 00:10:04,交易成功,15.2,1,江苏省,2019-09
1,线上,小z杂货铺,愛hya爱,2019-09-01 00:14:52,交易成功,8.4,1,广东省,2019-09
2,线上,小z杂货铺,象95象大,2019-09-01 02:17:15,交易成功,8.4,1,辽宁省,2019-09
3,线上,小z杂货铺,卡哇伊氛十,2019-09-01 03:37:28,交易成功,22.0,1,广西壮族自治区,2019-09
4,线上,小z杂货铺,一只羊哈阿,2019-09-01 08:53:50,交易成功,85.0,1,辽宁省,2019-09


In [90]:
sample_c = sample_c.loc[sample_c['客户昵称'].isin(history['客户昵称']) == False,:]
print('2019年10月新增客户：',len(sample_c))
sample_c.head()

2019年10月新增客户： 7083


,客户昵称,付款时间
0,0000栗,1
1,000ab,1
2,000al,1
3,000bb,1
4,000il,1


In [91]:
re = []
for i in ['2019-11', '2019-12', '2020-01', '2020-02']:
    next_month = order.loc[order['时间标签'] == i,:]
    target_users = sample_c.loc[sample_c['客户昵称'].isin(next_month['客户昵称']) == True,:]
    re.append([i+'留存情况：',len(target_users)])
re

[['2019-11留存情况：', 539],
 ['2019-12留存情况：', 428],
 ['2020-01留存情况：', 414],
 ['2020-02留存情况：', 426]]

In [92]:
re.insert(0,['2019年10月新增客户：',len(sample_c)])
re

[['2019年10月新增客户：', 7083],
 ['2019-11留存情况：', 539],
 ['2019-12留存情况：', 428],
 ['2020-01留存情况：', 414],
 ['2020-02留存情况：', 426]]

### 完整代码

In [123]:
#引入时间标签
month_lst = order['时间标签'].unique()
final = pd.DataFrame()

for i in range(len(month_lst) - 1):
    
    #构造和月份一样长的列表，方便后续格式统一
    count = [0] * len(month_lst)
    
    #筛选出当月订单，并按客户昵称分组
    target_month = order.loc[order['时间标签'] == month_lst[i],:]
    target_users = target_month.groupby('客户昵称')['支付金额'].sum().reset_index()
    
    #如果是第一个月份，则跳过（因为不需要和历史数据验证是否为新增客户）
    if i == 0:
        new_target_users = target_month.groupby('客户昵称')['支付金额'].sum().reset_index()
    else:
        #如果不是，找到之前的历史订单
        history = order.loc[order['时间标签'].isin(month_lst[:i]),:]
        #筛选出未在历史订单出现过的新增客户
        new_target_users = target_users.loc[target_users['客户昵称'].isin(history['客户昵称']) == False,:]
    
    #当月新增客户数放在第一个值中
    count[0] = len(new_target_users)
    
    #以月为单位，循环遍历，计算留存情况
    for j,ct in zip(range(i + 1,len(month_lst)),range(1,len(month_lst))):
        #下一个月的订单
        next_month = order.loc[order['时间标签'] == month_lst[j],:]
        next_users = next_month.groupby('客户昵称')['支付金额'].sum().reset_index()
        #计算在该月仍然留存的客户数量
        isin = new_target_users['客户昵称'].isin(next_users['客户昵称']).sum()
        count[ct] = isin
        
    #格式转置
    result = pd.DataFrame({month_lst[i]:count}).T
    
    #合并
    final = pd.concat([final,result])

final.columns = ['当月新增','+1月','+2月','+3月','+4月','+5月']

In [124]:
final

,当月新增,+1月,+2月,+3月,+4月,+5月
2019-09,2042,253,219,167,159,165
2019-10,7083,539,428,414,426,0
2019-11,4744,448,370,294,0,0
2019-12,4983,491,340,0,0,0
2020-01,5126,484,0,0,0,0


In [125]:
result = final.divide(final['当月新增'],axis = 0).iloc[:,1:]
result['当月新增'] = final['当月新增']
result

,+1月,+2月,+3月,+4月,+5月,当月新增
2019-09,0.123898,0.107248,0.081783,0.077865,0.080803,2042
2019-10,0.076098,0.060426,0.058450,0.060144,0.000000,7083
2019-11,0.094435,0.077993,0.061973,0.000000,0.000000,4744
2019-12,0.098535,0.068232,0.000000,0.000000,0.000000,4983
2020-01,0.094421,0.000000,0.000000,0.000000,0.000000,5126
